In [1]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)

#modeling

from xgboost import XGBRegressor 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, train_test_split, KFold, StratifiedGroupKFold, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn import ensemble
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [38]:
# LSTM modeling
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from pandas import DataFrame , concat
from sklearn.metrics import mean_absolute_error , mean_squared_error
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from numpy import mean , concatenate
from math import sqrt
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation
from numpy import array , hstack
from tensorflow import keras
import tensorflow as tf
# if you're using google drive to store the dataset
#from google.colab import drive

In [2]:
data_path = '/global/scratch/users/yaqiantang/discoveryData/'

# 5th Site - US-Var

## Data Pre-processing

In [45]:
site_id = 'US-Var' # 
df = pd.read_csv(data_path + 'data_full_half_hourly_raw_v0_1_' + site_id + '.csv')
df.head()

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,200011010000,200011010030,8.59,0,8.117,0.0,0.0,0,0.0,297.554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
1,200011010030,200011010100,8.54,0,7.898,0.0,0.0,0,0.0,297.554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
2,200011010100,200011010130,8.51,0,7.680,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
3,200011010130,200011010200,8.07,0,7.353,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
4,200011010200,200011010230,7.43,0,7.027,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate


In [46]:
df.columns

Index(['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F', 'TA_F_QC', 'TA_ERA',
       'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC', 'SW_IN_ERA', 'LW_IN_F',
       'LW_IN_F_QC', 'LW_IN_ERA', 'VPD_F', 'VPD_F_QC', 'VPD_ERA', 'P_F',
       'P_F_QC', 'P_ERA', 'PA_F', 'PA_F_QC', 'PA_ERA', 'NETRAD', 'PPFD_IN',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF', 'datetime',
       'year', 'month', 'day', 'hour', 'SITE_ID', 'date', 'NEE_VUT_REF_qa',
       'SW_DIF', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'koppen'],
      dtype='object')

In [47]:
target = 'GPP_NT_VUT_REF'
features = ['year','TA_ERA','SW_IN_ERA','P_ERA','VPD_ERA','b1','b2','b3','b4','b5','b6',
            'b7','NDVI','EVI','NIRv']
# and categorical features: IGBP and koppen

df['datetime'] = pd.to_datetime(df['datetime'])
df = df.set_index('datetime')
df.head()

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
datetime,,,,,,,,,,,,,,,,,,,,,
2000-11-01 00:00:00,200011010000,200011010030,8.59,0,8.117,0.0,0.0,0,0.0,297.554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
2000-11-01 00:30:00,200011010030,200011010100,8.54,0,7.898,0.0,0.0,0,0.0,297.554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
2000-11-01 01:00:00,200011010100,200011010130,8.51,0,7.680,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
2000-11-01 01:30:00,200011010130,200011010200,8.07,0,7.353,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
2000-11-01 02:00:00,200011010200,200011010230,7.43,0,7.027,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate


In [48]:
# One hot encoding categorical variables: IGBP and Koppen
df_ohe = df[[target, 'IGBP', 'koppen'] + features]

categorical_columns = ['IGBP', 'koppen']
for col in categorical_columns:
    col_ohe = pd.get_dummies(df[col], prefix=col)
    df_ohe = pd.concat((df_ohe, col_ohe), axis=1).drop(col, axis=1)

df_ohe.columns

Index(['GPP_NT_VUT_REF', 'year', 'TA_ERA', 'SW_IN_ERA', 'P_ERA', 'VPD_ERA',
       'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'NDVI', 'EVI', 'NIRv',
       'IGBP_GRA', 'koppen_Temperate'],
      dtype='object')

In [49]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    return df

In [50]:
# clean the data.
df = clean_dataset(df_ohe)
df.head()

,GPP_NT_VUT_REF,year,TA_ERA,SW_IN_ERA,P_ERA,VPD_ERA,b1,b2,b3,b4,b5,b6,b7,NDVI,EVI,NIRv,IGBP_GRA,koppen_Temperate
datetime,,,,,,,,,,,,,,,,,,
2001-01-01 00:00:00,-1.84180,2001,2.171,0.0,0.0,0.893,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1
2001-01-01 00:30:00,-1.91023,2001,2.161,0.0,0.0,1.107,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1
2001-01-01 01:00:00,-1.85277,2001,2.150,0.0,0.0,1.321,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1
2001-01-01 01:30:00,-5.84140,2001,2.044,0.0,0.0,1.391,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1
2001-01-01 02:00:00,-2.51478,2001,1.938,0.0,0.0,1.461,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1


In [51]:
# split the dataset into training and testing
Data = df[features + [target]]

In [52]:
import binascii
def test_set_check(identifier, test_ratio):
    return binascii.crc32(np.int64(hash(identifier))) & 0xffffffff < test_ratio * 2**32

# Function to split train/test
def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    print(ids)
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [53]:
test, train = split_train_test_by_id(Data, 0.7, "year")
print(train.shape, test.shape)

datetime
2001-01-01 00:00:00    2001
2001-01-01 00:30:00    2001
2001-01-01 01:00:00    2001
2001-01-01 01:30:00    2001
2001-01-01 02:00:00    2001
                       ... 
2014-12-31 21:30:00    2014
2014-12-31 22:00:00    2014
2014-12-31 22:30:00    2014
2014-12-31 23:00:00    2014
2014-12-31 23:30:00    2014
Name: year, Length: 238752, dtype: int64
(153936, 16) (84816, 16)


In [54]:
train = train[features + [target]]

test = test[features + [target]]

data = Data[features + [target]]

In [55]:
scaler = MinMaxScaler(feature_range=(0, 1))
data = np.vstack((train, test))
data_scaled = scaler.fit_transform(data)
train_scaled = data_scaled[:11116,]
test_scaled = data_scaled[11116:,]

In [40]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
  # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
  # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
    X.append(seq_x)
    y.append(seq_y)
    return array(X), array(y)

# choose a number of time steps #change this accordingly
n_steps_in, n_steps_out = 12, 12
# covert into input/output
train_X, train_y = split_sequences(train_scaled, n_steps_in, n_steps_out)
test_X, test_y = split_sequences(test_scaled, n_steps_in, n_steps_out)

print ("train_X.shape" , train_X.shape) 
print ("train_y.shape" , train_y.shape)
print ("test_X.shape" , test_X.shape) 
print ("test_y.shape" , test_y.shape)

train_X.shape (1, 12, 15)
train_y.shape (1, 11)
test_X.shape (1, 12, 15)
test_y.shape (1, 11)


## Time Series LSTM

In [65]:
#optimizer learning rate
opt = keras.optimizers.Adam(learning_rate=0.0001)
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_in, 15)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(n_steps_out))
model.add(Activation('linear'))
model.compile(loss='mse' , optimizer=opt , metrics=['mse'])

In [66]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 12, 15)            1860      
                                                                 
 lstm_13 (LSTM)              (None, 15)                1860      
                                                                 
 dense_6 (Dense)             (None, 12)                192       
                                                                 
 activation_6 (Activation)   (None, 12)                0         
                                                                 
Total params: 3,912
Trainable params: 3,912
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2)

Epoch 1/100


ValueError: in user code:

    File "/global/scratch/users/yaqiantang/conda/ml/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/global/scratch/users/yaqiantang/conda/ml/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/global/scratch/users/yaqiantang/conda/ml/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/global/scratch/users/yaqiantang/conda/ml/lib/python3.10/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/global/scratch/users/yaqiantang/conda/ml/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/global/scratch/users/ya

In [ ]:
trainPredict = model.predict(train_X)
testPredict = model.predict(test_X)

In [ ]:
trainScore = np.sqrt(mean_squared_error(train_y, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(test_y, testPredict))
print('Test Score: %.2f RMSE' % (testScore))